In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap

In [2]:
data = pd.read_csv('./redacted-Results-Oct20-Mar21.csv')
loc = pd.read_csv('liquor_location_1.csv').rename(columns = {'Zip Code' : 'zip', '??(????)' : 'latitude', '??(????).1' : 'longitude'})

In [3]:
data['No of months of assistance'] = data['No of months of assistance'].fillna(1)
data['RRF Award Amount '] = data['RRF Award Amount '].fillna(0)

In [4]:
def to_num(column):
    p = []
    for i in range(0, len(column)):
        a = column.to_numpy()
        p.append(float(str(a[i]).replace(',', '')))
    return p


In [5]:
data['RRF Award Amount '] = to_num(data['RRF Award Amount '])

In [6]:
data['No of months of assistance']  = to_num(data['No of months of assistance'])

In [7]:
data['Monthly average RRF'] = data['RRF Award Amount '] / data['No of months of assistance']

In [11]:
data.head(1)

,id,notes,original_zip,manual_ zip,match_kind,state_code,county_code,tract_code,block_code,RRF Award Amount,...,RRF Award Type,Mthly Self Employment,Mthly Unemployment,Mthly Interest or Dividend,Mthly Pension or Retirement,Mthly TANF/Public Assistance,Mthly Alimony income,Mthly Armed Forces income,Mthly Other income,Monthly average RRF
0,7.0,NaN,2119.0,NaN,Exact,25.0,25.0,81400.0,3003.0,6000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6000.0


In [9]:
loc.head(1)

,zip,Counts,latitude,longitude
0,2467,4,42.3167,-71.1605


In [12]:
def zip(col):
    p = []
    a = col.to_numpy()
    for i in range(0, len(a)):
        x = '0'+str(a[i]).replace('.0', '')
        p.append(x)
    return p
    

In [13]:
data['original_zip'] = data['original_zip'].fillna('')

In [14]:
data['original_zip'] = zip(data['original_zip'])

In [15]:
data = data.rename(columns={'original_zip': 'zip'})

In [16]:
loc['zip'] = zip(loc['zip'])

In [17]:
data_loc = pd.merge(data, loc, how = 'left', on = 'zip')
data_loc = data_loc.fillna(0)
data_loc.head(1)

,id,notes,zip,manual_ zip,match_kind,state_code,county_code,tract_code,block_code,RRF Award Amount,...,Mthly Interest or Dividend,Mthly Pension or Retirement,Mthly TANF/Public Assistance,Mthly Alimony income,Mthly Armed Forces income,Mthly Other income,Monthly average RRF,Counts,latitude,longitude
0,7.0,0.0,02119,0.0,Exact,25.0,25.0,81400.0,3003.0,6000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6000.0,24.0,42.324,-71.085


In [18]:
data_loc['total_aids'] = data_loc['RRF Award Amount ']+data_loc['Mthly Self Employment']+data_loc['Mthly Unemployment']+data_loc['Mthly Interest or Dividend']+data_loc['Mthly Pension or Retirement']+data_loc['Mthly TANF/Public Assistance']+data_loc['Mthly Alimony income']+data_loc['Mthly Armed Forces income']+data_loc['Mthly Other income']

In [19]:
data_loc

,id,notes,zip,manual_ zip,match_kind,state_code,county_code,tract_code,block_code,RRF Award Amount,...,Mthly Pension or Retirement,Mthly TANF/Public Assistance,Mthly Alimony income,Mthly Armed Forces income,Mthly Other income,Monthly average RRF,Counts,latitude,longitude,total_aids
0,7.0,0.0,02119,0.0,Exact,25.0,25.0,81400.0,3003.0,6000.00,...,0.0,0.0,0.0,0.0,0.0,6000.000000,24.0,42.3240,-71.0850,6000.00
1,8.0,0.0,02124,0.0,Exact,25.0,25.0,100300.0,3005.0,5700.00,...,0.0,0.0,0.0,0.0,0.0,1900.000000,37.0,42.2858,-71.0718,7840.00
2,56.0,0.0,02119,0.0,Exact,25.0,25.0,80100.0,1051.0,3750.00,...,0.0,0.0,0.0,0.0,0.0,625.000000,24.0,42.3240,-71.0850,3750.00
3,63.0,0.0,02128,0.0,Exact,25.0,25.0,51200.0,3000.0,4000.00,...,0.0,0.0,530.0,0.0,0.0,4000.000000,131.0,42.3719,-71.0151,4530.00
4,75.0,0.0,02128,0.0,Exact,25.0,25.0,50101.0,3001.0,6000.00,...,0.0,0.0,0.0,0.0,0.0,6000.000000,131.0,42.3719,-71.0151,6800.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,9192.0,0.0,02113,0.0,0,25.0,25.0,30400.0,3003.0,4000.00,...,0.0,0.0,0.0,0.0,0.0,4000.000000,89.0,42.3643,-71.0575,5000.00
750,9219.0,0.0,02127,0.0,0,25.0,25.0,60501.0,3003.0,2032.07,...,0.0,0.0,0.0,0.0,0.0,2032.070000,79.0,42.3359,-71.0421,4332.07
751,9419.0,0.0,02128,0.0,0,25.0,25.0,50600.0,2007.0,4000.00,...,0.0,0.0,0.0,0.0,0.0,1000.000000,131.0,42.3719,-71.0151,4000.00
752,9446.0,0.0,02120,0.0,0,25.0,25.0,81102.0,1001.0,6000.00,...,0.0,0.0,0.0,0.0,0.0,1500.000000,14.0,42.3326,-71.0957,6300.00


In [32]:
num_of_rrf_to_zip = data_loc.groupby('zip').count().reset_index()[['zip', 'Counts']].sort_values('Counts', ascending = False)
num_of_rrf_to_zip.to_excel('./num_of_rrf_to_zip.xlsx')

In [26]:
num_of_rrf_to_zip = data_loc.groupby('zip').sum()
num_of_rrf_to_zip.head(1)

,id,notes,manual_ zip,state_code,county_code,tract_code,block_code,RRF Award Amount,No of months of assistance,Rent owing,...,Mthly Pension or Retirement,Mthly TANF/Public Assistance,Mthly Alimony income,Mthly Armed Forces income,Mthly Other income,Monthly average RRF,Counts,latitude,longitude,total_aids
zip,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
total_owning_to_zip = data_loc.groupby('zip').sum().reset_index()[['zip', 'Rent owing']].sort_values('Rent owing', ascending = False)
total_owning_to_zip.to_excel('./total_owed_to_zip_MAR.xlsx')

In [33]:
num_of_rrf_to_zip

,zip,Counts
17,02128,181
13,02124,82
14,02125,52
9,02119,49
12,02122,46
11,02121,43
24,02136,37
15,02126,36
23,02135,31
20,02131,23


In [38]:
pp = pd.merge(num_of_rrf_to_zip, total_owning_to_zip, on = 'zip')
pp['Average owe'] = pp['Rent owing'] / pp['Counts']
pp.to_excel('./Average_owe_to_zip_MAR.xlsx')

In [241]:
RRF = data_loc[['latitude', 'longitude', 'RRF Award Amount ']]
RRF_list = RRF.dropna().values.tolist()
m = folium.Map(location = [42.3601, -71.0589],
               zoom_start = 12)
HeatMap(RRF_list).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
m.save('RRF.html')

In [242]:
RRF_monthly = data_loc[['latitude', 'longitude', 'Monthly average RRF']]
RRF_monthly_list = RRF_monthly.dropna().values.tolist()
m = folium.Map(location = [42.3601, -71.0589],
               zoom_start = 12)
HeatMap(RRF_monthly_list).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
m.save('RRF_monthly.html')